In [1]:
import pandas as pd

TESLA = '/kaggle/input/tesla-stock-data-updated-till-28jun2021/TSLA.csv'

df = pd.read_csv(filepath_or_buffer=TESLA, parse_dates=['Date'])
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,3.800,5.000,3.508,4.778,4.778,93831500
1,2010-06-30,5.158,6.084,4.660,4.766,4.766,85935500
2,2010-07-01,5.000,5.184,4.054,4.392,4.392,41094000
3,2010-07-02,4.600,4.620,3.742,3.840,3.840,25699000
4,2010-07-06,4.000,4.000,3.166,3.222,3.222,34334500


In [2]:
from plotly.express import line
line(data_frame=df, x='Date', y=['Open', 'High', 'Low', 'Close', 'Adj Close'], log_y=True)

In [3]:
# total return
total_return = 100 * df['Adj Close'].values[-1] / df['Adj Close'].values[0]
print('total return: {} pct'.format(round(100 * total_return)/100))

total return: 21220.59 pct


In [4]:
# years
years = (df['Date'].max() - df['Date'].min()).days / 365.25
print('years: {}'.format(round(100 * years)/100))

years: 11.73


In [5]:
from math import log
from math import exp

def get_rate(start_value: float, end_value: float, years: float) -> float:
    rate = exp(1.0/years * log(end_value/start_value)) - 1.0
    return rate

annualized_rate = get_rate(start_value=df['Adj Close'].values[0], end_value=df['Adj Close'].values[-1], years=years)

print('annualized pct rate: {}'.format(100 * round(10000 * annualized_rate) / 10000))

annualized pct rate: 57.86


In [6]:
total = 4.778
rate = 0.58
for year in range(1, 13):
    total = (1 + rate) * total 
    print(year, total)

1 7.549239999999999
2 11.927799199999999
3 18.845922736
4 29.77655792288
5 47.0469615181504
6 74.33419919867762
7 117.44803473391065
8 185.56789487957883
9 293.1972739097346
10 463.25169277738064
11 731.9376745882614
12 1156.461525849453
